# Estimating Work Tour Scheduling

This notebook illustrates how to re-estimate the mandatory tour scheduling component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import os
import larch  # !conda install larch -c conda-forge # for estimation
import pandas as pd

JAX not found. Some functionality will be unavailable.


We'll work in our `test` directory, where ActivitySim has saved the estimation data bundles.

In [2]:
os.chdir('test')

# Load data and prep model for estimation

In [3]:
modelname = "mandatory_tour_scheduling_work"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)

loading from output/estimation_data_bundle/mandatory_tour_scheduling_work/tour_scheduling_work_coefficients.csv
loading from output/estimation_data_bundle/mandatory_tour_scheduling_work/mandatory_tour_scheduling_work_SPEC.csv
loading from output/estimation_data_bundle/mandatory_tour_scheduling_work/mandatory_tour_scheduling_work_alternatives_combined.parquet
loading from output/estimation_data_bundle/mandatory_tour_scheduling_work/mandatory_tour_scheduling_work_choosers_combined.parquet


/Users/jpn/Git/est-mode/.env/ESTER/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


In [4]:
import numpy as np

j = np.isnan(model.datatree.root_dataset['mode_choice_logsum'])
j.drop_vars(j.coords)

model.datatree.root_dataset.assign(mode_choice_logsum_missing=j)


<xarray.Dataset> Size: 72MB
Dimensions:                                                                              (
                                                                                          tour_id: 2282,
                                                                                          tdd: 190)
Coordinates:
  * tour_id                                                                              (tour_id) int64 18kB ...
  * tdd                                                                                  (tdd) int64 2kB ...
Data variables: (12/112)
    start                                                                                (tour_id, tdd) int8 434kB ...
    end                                                                                  (tour_id, tdd) int8 434kB ...
    duration                                                                             (tour_id, tdd) int8 434kB ...
    out_period                                                                           (tour_id, tdd) object 3MB ...
    in_period                                                                            (tour_id, tdd) object 3MB ...
    mode_choice_logsum                                                                   (tour_id, tdd) float64 3MB ...
    ...                                                                                   ...
    school_zone_id                                                                       (tour_id) int64 18kB ...
    home_zone_id                                                                         (tour_id) int64 18kB ...
    start_previous                                                                       (tour_id) int8 2kB ...
    end_previous                                                                         (tour_id) int8 2kB ...
    override_choice_plus1                                                                (tour_id) int64 18kB ...
    model_choice_plus1                                                                   (tour_id) int64 18kB ...
Attributes:
    _caseid_:  tour_id
    _altid_:   tdd

In [5]:
model.datatree.root_dataset

<xarray.Dataset> Size: 72MB
Dimensions:                                                                              (
                                                                                          tour_id: 2282,
                                                                                          tdd: 190)
Coordinates:
  * tour_id                                                                              (tour_id) int64 18kB ...
  * tdd                                                                                  (tdd) int64 2kB ...
Data variables: (12/112)
    start                                                                                (tour_id, tdd) int8 434kB ...
    end                                                                                  (tour_id, tdd) int8 434kB ...
    duration                                                                             (tour_id, tdd) int8 434kB ...
    out_period                                                                           (tour_id, tdd) object 3MB ...
    in_period                                                                            (tour_id, tdd) object 3MB ...
    mode_choice_logsum                                                                   (tour_id, tdd) float64 3MB ...
    ...                                                                                   ...
    school_zone_id                                                                       (tour_id) int64 18kB ...
    home_zone_id                                                                         (tour_id) int64 18kB ...
    start_previous                                                                       (tour_id) int8 2kB ...
    end_previous                                                                         (tour_id) int8 2kB ...
    override_choice_plus1                                                                (tour_id) int64 18kB ...
    model_choice_plus1                                                                   (tour_id) int64 18kB ...
Attributes:
    _caseid_:  tour_id
    _altid_:   tdd

# Review data loaded from the EDB

The next (optional) step is to review the EDB, including the coefficients, utilities specification, and chooser and alternative data.

## Coefficients

In [6]:
data.coefficients

,value,constrain
coefficient_name,,
coef_dummy,1.000000,T
coef_free_flow_round_trip_auto_time_shift_effects_departure,-0.001140,F
coef_free_flow_round_trip_auto_time_shift_effects_duration,0.002210,F
coef_part_time_worker_departure_shift_effects,0.067360,F
coef_non_working_adult_duration_shift_effects,-0.120700,F
...,...,...
coef_duration_constants_9_hours,0.055706,F
coef_duration_constants_10_hours,0.000000,T
coef_duration_constants_11_hours,-0.347795,F


## Utility specification

In [7]:
data.spec

,Label,Description,Expression,Coefficient
0,util_free_flow_round_trip_auto_time_shift_effe...,Free-flow round trip auto time shift effects -...,roundtrip_auto_time_to_work * start,coef_free_flow_round_trip_auto_time_shift_effe...
1,util_free_flow_round_trip_auto_time_shift_effe...,Free-flow round trip auto time shift effects -...,roundtrip_auto_time_to_work * duration,coef_free_flow_round_trip_auto_time_shift_effe...
2,util_part_time_worker_departure_shift_effects,Part-time worker departure shift effects,(ptype == 2) * start,coef_part_time_worker_departure_shift_effects
3,util_non_working_adult_duration_shift_effects,Non-working adult duration shift effects,(ptype == 4) * duration,coef_non_working_adult_duration_shift_effects
4,util_university_student_departure_shift_effects,University student departure shift effects,(ptype == 3) * start,coef_university_student_departure_shift_effects
...,...,...,...,...
60,util_duration_constants_9_hours,Duration Constants -- 9 hours,duration == 9,coef_duration_constants_9_hours
61,util_duration_constants_10_hours,Duration Constants -- 10 hours,duration == 10,coef_duration_constants_10_hours
62,util_duration_constants_11_hours,Duration Constants -- 11 hours,duration == 11,coef_duration_constants_11_hours
63,util_duration_constants_12_to_13_hours,Duration Constants -- 12 to 13 hours,(duration > 11) & (duration < 14),coef_duration_constants_12_to_13_hours


## Chooser data

In [8]:
data.chooser_data

,tour_id,model_choice,override_choice,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,...,home_is_rural,mandatory_tour_frequency,is_worker,is_student,is_university,workplace_zone_id,school_zone_id,home_zone_id,start_previous,end_previous
0,2974630,80,29,72551,work,1,1,1,1,mandatory,...,False,work1,True,False,False,5,-1,72,5,5
1,3129077,47,79,76318,work,1,1,1,1,mandatory,...,False,work1,True,False,False,142,-1,248,5,5
2,3339325,7,42,81446,work,2,1,1,2,mandatory,...,False,work2,True,False,False,582,-1,541,5,5
3,3339326,139,138,81446,work,2,2,2,2,mandatory,...,False,work2,True,False,False,582,-1,541,5,12
4,3402916,50,10,82997,work,1,1,1,1,mandatory,...,False,work1,True,False,False,435,-1,652,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,308290847,60,27,7519288,work,1,1,1,1,mandatory,...,False,work1,True,False,False,873,-1,871,5,5
2278,308352511,47,32,7520792,work,1,1,1,1,mandatory,...,False,work1,True,False,False,1019,-1,1008,5,5
2279,308380719,34,11,7521480,work,1,1,1,1,mandatory,...,False,work1,True,False,False,972,-1,1008,5,5
2280,308393634,58,31,7521795,work,1,1,1,1,mandatory,...,False,work1,True,False,False,971,-1,1047,5,5


## Alternatives data

In [9]:
data.alt_values

,tour_id,start,end,duration,tdd,out_period,in_period,mode_choice_logsum,util_free_flow_round_trip_auto_time_shift_effects_departure,util_free_flow_round_trip_auto_time_shift_effects_duration,...,util_arrival_constants_late,util_duration_constants_0_to_2_hours,util_duration_constants_3_to_4_hours,util_duration_constants_5_to_6_hours,util_duration_constants_7_to_8_hours,util_duration_constants_9_hours,util_duration_constants_10_hours,util_duration_constants_11_hours,util_duration_constants_12_to_13_hours,util_duration_constants_14_to_18_hours
0,2974630,5,5,0,0,EA,EA,2.979801,62.449997,0.000000,...,False,True,False,False,False,False,False,False,False,False
1,2974630,5,6,1,1,EA,AM,3.166835,62.449997,12.490000,...,False,True,False,False,False,False,False,False,False,False
2,2974630,5,7,2,2,EA,AM,3.166835,62.449997,24.980000,...,False,True,False,False,False,False,False,False,False,False
3,2974630,5,8,3,3,EA,AM,3.166835,62.449997,37.470001,...,False,False,True,False,False,False,False,False,False,False
4,2974630,5,9,4,4,EA,AM,3.166835,62.449997,49.959999,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425656,309081532,21,22,1,185,EV,EV,4.159152,429.870026,20.470001,...,True,True,False,False,False,False,False,False,False,False
425657,309081532,21,23,2,186,EV,EV,4.159152,429.870026,40.940002,...,True,True,False,False,False,False,False,False,False,False
425658,309081532,22,22,0,187,EV,EV,4.159152,450.340027,0.000000,...,True,True,False,False,False,False,False,False,False,False
425659,309081532,22,23,1,188,EV,EV,4.159152,450.340027,20.470001,...,True,True,False,False,False,False,False,False,False,False


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [10]:
model.doctor(repair_ch_av="-")

problem: chosen_but_not_available has (18 issues)


(<larch.Model (MNL) "None">,
 ┣ chosen_but_not_available:    altid  n example rows
 ┃                           0     55  1         1136
 ┃                           1    100  1          522
 ┃                           2    105  1          140
 ┃                           3    106  1         1369
 ┃                           4    115  1         2029
 ┃                           5    117  1          711
 ┃                           6    125  1          900
 ┃                           7    126  1          186
 ┃                           8    128  2     218, 772
 ┃                           9    131  1          963
 ┃                           10   137  1         2109
 ┃                           11   141  2     166, 873
 ┃                           12   143  1          351
 ┃                           13   147  1         1742
 ┃                           14   150  1         1484
 ┃                           15   155  1         1825
 ┃                           16   160  1         1117

In [11]:
model.estimate(maxiter=900)

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction,3.435663,3.435663,0.362700,-25.0,25.0,0.0,0
coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction,-0.594068,-0.594068,-0.101200,-25.0,25.0,0.0,0
coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction,0.772027,0.772027,0.177100,-25.0,25.0,0.0,0
coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction,-4.580797,-4.580797,-0.212300,-25.0,25.0,0.0,0
coef_arrival_constants_am_peak,-1.944152,-1.944152,-1.854521,-25.0,25.0,0.0,0
...,...,...,...,...,...,...,...
coef_subsequent_of_2plus_work_tours_duration_lt_8_hrs,13.264625,13.264625,2.582000,-25.0,25.0,0.0,0
coef_subsequent_tour_must_start_after_previous_tour_ends,-100.000000,-100.000000,-100.000000,-100.0,-100.0,0.0,1
coef_tours_by_student_duration_lt_8_hrs,4.383956,4.383956,2.582000,-25.0,25.0,0.0,0


/Users/jpn/Git/est-mode/larch/src/larch/model/jaxmodel.py:1156: PossibleOverspecification: Model is possibly over-specified (hessian is nearly singular).
  self.calculate_parameter_covariance()


┣                x: coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction               3.435663
┃                   coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction        -0.594068
┃                   coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction            0.772027
┃                   coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction     -4.580797
┃                   coef_arrival_constants_am_peak                                                          -1.944152
┃                                                                                                             ...    
┃                   coef_subsequent_of_2plus_work_tours_duration_lt_8_hrs                                   13.264625
┃                   coef_subsequent_tour_must_start_after_previous_tour_ends                              -100.000000
┃                   coef_tours_by_student_duration_lt_8_hrs                                                  4.383956
┃                   coef_tours_by_worker_duration_lt_8_hrs                                                   2.714556
┃                   coef_university_student_departure_shift_effects                                          0.014618
┃                   Length: 63, dtype: float64
┣          logloss: 3.9514412791246034
┣        d_logloss: coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction             2.563365e-06
┃                   coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction       9.957898e-05
┃                   coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction          6.243871e-06
┃                   coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction   -1.823687e-04
┃                   coef_arrival_constants_am_peak                                                         3.368144e-05
┃                                                                                                              ...     
┃                   coef_subsequent_of_2plus_work_tours_duration_lt_8_hrs                                  5.787341e-09
┃                   coef_subsequent_tour_must_start_after_previous_tour_ends                               0.000000e+00
┃                   coef_tours_by_student_duration_lt_8_hrs                                                9.024491e-07
┃                   coef_tours_by_worker_duration_lt_8_hrs                                                 9.024491e-07
┃                   coef_university_student_departure_shift_effects                                       -2.760222e-05
┃                   Length: 63, dtype: float64
┣              nit: 206
┣             nfev: 222
┣             njev: 206
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=16, microseconds=666673)
┣           method: 'slsqp'
┣          n_cases: 2282
┣ iteration_number: 206
┣          loglike: -8938.160173379852

### Estimated coefficients

In [12]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value,Constrained
Parameter,,,,,,
coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction,3.44,13.1,0.26,,0.00,
coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction,-0.594,0.617,-0.96,,0.00,
coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction,0.772,0.459,1.68,,0.00,
coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction,-4.58,20.2,-0.23,,0.00,
coef_arrival_constants_am_peak,-1.94,0.509,-3.82,***,0.00,
coef_arrival_constants_early,0.00,0.00,NA,,0.00,fixed value
coef_arrival_constants_evening,-0.0913,0.205,-0.45,,0.00,
coef_arrival_constants_late,-1.09,0.271,-4.01,***,0.00,
coef_arrival_constants_midday_1,-0.263,0.193,-1.36,,0.00,


# Output Estimation Results

In [13]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [14]:
model.to_xlsx(
    result_dir/f"{modelname}_model_estimation.xlsx", 
    data_statistics=False,
)

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [15]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_dummy,1.000000,T
1,coef_free_flow_round_trip_auto_time_shift_effe...,-0.001483,F
2,coef_free_flow_round_trip_auto_time_shift_effe...,0.001780,F
3,coef_part_time_worker_departure_shift_effects,0.052073,F
4,coef_non_working_adult_duration_shift_effects,-0.120700,F
...,...,...,...
59,coef_duration_constants_9_hours,-0.045027,F
60,coef_duration_constants_10_hours,0.000000,T
61,coef_duration_constants_11_hours,-0.222485,F
62,coef_duration_constants_12_to_13_hours,-0.782156,F
